In [1]:
from datasets import load_dataset
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
import numpy as np
!pip install evaluate
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


In [2]:
# Load dataset
dataset = load_dataset('imdb')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
# Split the training set into train/validation
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]      # 22,500 samples
val_dataset   = split_dataset["test"]

In [4]:
# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset   = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [6]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Define metrics
accuracy = evaluate.load('accuracy')

In [8]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return accuracy.compute(predictions=predictions, references=labels)

In [9]:
# Training arguments
training_args = TrainingArguments(
  report_to='none', # wandb issue
  output_dir='./results',
  per_device_train_batch_size=32, #larger batch for bigger accelerator
  num_train_epochs=3,
  learning_rate=2e-5,
  weight_decay=0.01,
  eval_strategy='epoch',
  )


#setup trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-530630015.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.262600,0.219235,0.921600
2,0.163800,0.269968,0.918800
3,0.076300,0.251423,0.936000


TrainOutput(global_step=2112, training_loss=0.1506632101355177, metrics={'train_runtime': 1343.6806, 'train_samples_per_second': 50.235, 'train_steps_per_second': 1.572, 'total_flos': 1.77599962368e+16, 'train_loss': 0.1506632101355177, 'epoch': 3.0})

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import os
os.listdir("./drive/MyDrive/Colab Notebooks/leos stuff/bert-base-finetuned")

[]

In [13]:
save_path = "./drive/MyDrive/Colab Notebooks/leos stuff/bert-base-finetuned"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

('./drive/MyDrive/Colab Notebooks/leos stuff/tokenizer_config.json',
 './drive/MyDrive/Colab Notebooks/leos stuff/special_tokens_map.json',
 './drive/MyDrive/Colab Notebooks/leos stuff/vocab.txt',
 './drive/MyDrive/Colab Notebooks/leos stuff/added_tokens.json',
 './drive/MyDrive/Colab Notebooks/leos stuff/tokenizer.json')